In [1]:
from pathlib import Path
import duckdb
from dcpy.utils import duckdb as dcpduckdb
import pandas as pd

pd.set_option('display.max_columns', 50)

In [2]:
DB_PATH = Path("facdb_qa.db")

In [16]:
# delete the database if it exists
DB_PATH.unlink(missing_ok=True)

# # create the database with extensions and credentials
# with duckdb.connect(str(DB_PATH)) as connection:
#     connection.sql(f"INSTALL spatial")
#     connection.sql(f"LOAD spatial")


In [17]:
# test
with duckdb.connect(str(DB_PATH)) as connection:
    dcpduckdb.setup_s3_secret(DB_PATH)
    connection.sql(
        "DESCRIBE TABLE 's3://edm-recipes/datasets/test_nypl_libraries/20240124/test_nypl_libraries.csv'"
    ).show()

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ field_1     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ lon         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ lat         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ name        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ zipcode     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ address     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ locality    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ region      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘



In [21]:
with duckdb.connect(str(DB_PATH)) as connection:
    dcpduckdb.setup_s3_secret(DB_PATH)
    connection.sql(
        """
            create table dpr_parksproperties_all as
            select * from
            read_csv(
                's3://edm-recipes/datasets/dpr_parksproperties/*/dpr_parksproperties.csv',
                union_by_name = true,
                filename = true
            )
        """
    )

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
with duckdb.connect(str(DB_PATH)) as connection:
    connection.sql("SHOW ALL TABLES").show()

┌──────────┬─────────┬─────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│ database │ schema  │          name           │                                                                                                                                                                                          column_na

In [25]:
with duckdb.connect(str(DB_PATH)) as connection:
    connection.sql("describe table dpr_parksproperties_all").show(max_rows=100)

┌─────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name   │ column_type │  null   │   key   │ default │  extra  │
│     varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ jurisdiction    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ mapped          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ zipcode         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ acres           │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ location        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ nys_assembly    │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ councildistrict │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ url             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ typecategory    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ us_congress     │ BIGINT      │ YES 

In [26]:
with duckdb.connect(str(DB_PATH)) as connection:
    connection.sql("select * from dpr_parksproperties_all").show()

┌──────────────┬─────────┬──────────────────────┬────────┬──────────────────────┬──────────────┬─────────────────┬──────────────────────┬──────────────────────┬─────────────┬──────────────────────┬──────────┬────────────┬─────────┬─────────────────────┬────────────────┬──────────┬──────────────────────┬──────────────────────┬────────────┬─────────────┬──────────────────────┬──────────┬─────────┬─────────────────────┬───────────┬───────────┬──────────────────────┬──────────────────────┬──────────────┬─────────┬────────────┬────────────────┬─────────┬────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [35]:
with duckdb.connect(str(DB_PATH)) as connection:
    # connection.sql("alter table dpr_parksproperties_all add column version varchar")
    connection.sql("update dpr_parksproperties_all set version = string_split(parse_dirpath(filename), '/')[-1]")
    connection.sql("select version, filename, location, from dpr_parksproperties_all").show()

┌──────────┬────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────┐
│ version  │                                    filename                                    │                           location                            │
│ varchar  │                                    varchar                                     │                            varchar                            │
├──────────┼────────────────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────┤
│ 20210123 │ s3://edm-recipes/datasets/dpr_parksproperties/20210123/dpr_parksproperties.csv │ Gravesend Bay, Bay 44 St. to Bay 49 St., Shore Pkwy.          │
│ 20210123 │ s3://edm-recipes/datasets/dpr_parksproperties/20210123/dpr_parksproperties.csv │ Atlantic Ave., Columbia Pl., State St.                        │
│ 20210123 │ s3://edm-recipes/datasets/dpr_parksprop

In [ ]:
old_dpr_parksproperties = duckdb.sql("select * from 's3://edm-recipes/datasets/dpr_parksproperties/20240109/dpr_parksproperties.csv'").df()
old_dpr_parksproperties

In [ ]:
new_dpr_parksproperties = duckdb.sql("select * from 's3://edm-recipes/datasets/dpr_parksproperties/20240814/dpr_parksproperties.csv'").df()
new_dpr_parksproperties

## `typecategory`

In [ ]:
new_dpr_parksproperties.compare(old_dpr_parksproperties)